In [1]:
import tensorflow as tf
import random, os, glob
import numpy as np
from PIL import Image
from keras.utils import np_utils
import pandas as pd

from sklearn.model_selection import train_test_split

tf.set_random_seed(42)  # reproducibility
nb_classes = 2

# load mnist image : 28 x 28

learning_rate = 0.001
training_epochs = 15
batch_size = 128

Using TensorFlow backend.


In [2]:
# total appx 1999
MAL = 2667 
# total appx 4000
BEN = 5253

TEST=1979

In [3]:
b = glob.glob('../image/0-normal/*x.png')

# benign : 4000
X_data_normal = np.empty((BEN, 28, 28, 1), dtype = "float32")
y_data_normal = np.empty((BEN,), dtype = "uint8")
cnt = 0

for i in b:
    im = Image.open(i).convert("L")
    out = im.resize((28,28))
    X_data_normal[cnt,:,:,0] = out
    y_data_normal[cnt,] = 0 # normal
    
    cnt = cnt+1
    if cnt == (BEN):
        break


m = glob.glob('../image/1-malware/*x.png')

# malware : 1999
X_data_malware = np.empty((MAL, 28, 28, 1), dtype = "float32")
y_data_malware = np.empty((MAL,), dtype = "uint8")
cnt = 0

for i in m:
    im = Image.open(i).convert("L")
    out = im.resize((28,28))
    X_data_malware[cnt,:,:,0] = out
    y_data_malware[cnt,] = 1 # malware
    
    cnt = cnt+1
    if cnt == (MAL):
        break

t = glob.glob('../image/second/*x.png')

injected_X_test = np.empty((TEST, 28, 28, 1), dtype = "float32")
injected_y_test = np.empty((TEST,), dtype = "uint8")

cnt = 0

for i in t:
    im = Image.open(i).convert("L")
    out = im.resize((28,28))

    injected_X_test[cnt,:,:,0] = out
    cnt = cnt+1
    if cnt == (TEST):
        break
        
X_train_benign, X_test_benign = train_test_split(X_data_normal,test_size=0.1)
y_train_benign, y_test_benign = train_test_split(y_data_normal,test_size=0.1)
X_train_malware, X_test_malware = train_test_split(X_data_malware,test_size=0.1)
y_train_malware, y_test_malware = train_test_split(y_data_malware,test_size=0.1)

In [4]:
print(len(b),len(m))

5253 2667


In [5]:
print(X_data_normal.shape,y_data_normal.shape,X_data_malware.shape,y_data_malware.shape)

(5253, 28, 28, 1) (5253,) (2667, 28, 28, 1) (2667,)


In [6]:
BEN_TRAIN=len(X_train_benign)
BEN_TEST=len(X_test_benign)
MAL_TRAIN=len(X_train_malware)
MAL_TEST=len(X_test_malware)

In [7]:
X_train = np.empty(((BEN_TRAIN+MAL_TRAIN), 28, 28, 1), dtype = "float32")
y_train = np.empty(((BEN_TRAIN+MAL_TRAIN),), dtype = "uint8")
X_test = np.empty(((BEN_TEST+MAL_TEST),  28, 28, 1), dtype = "float32")
y_test = np.empty(((BEN_TEST+MAL_TEST),), dtype = "uint8")
blind_X_test = np.empty((4600, 28, 28, 1), dtype = "float32")
blind_y_test = np.empty((4600,), dtype = "uint8")

#y_train_benign = np.zeros(BEN_TRAIN,)
#y_test_benign = np.zeros(BEN_TEST,)

X_train = np.concatenate((X_train_benign, X_train_malware), axis=0)
y_train = np.append(y_train_benign, y_train_malware)
X_test = np.concatenate((X_test_benign, X_test_malware), axis=0)
y_test = np.append(y_test_benign, y_test_malware)

In [8]:
X_train = X_train.astype("float32")
#X_train /= 255
Y_train = np_utils.to_categorical(y_train, nb_classes)

X_test = X_test.astype("float32")
#X_test /= 255
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [9]:
from sklearn.utils import shuffle
X_, Y_ = shuffle(X_train, Y_train)

In [10]:
tf.reset_default_graph()
# hyper parameters
print ("X_train.shape() = ", X_train.shape)
print ("Y_train.shape() = ", Y_train.shape)
print ("X_test.shape() = ", X_test.shape)


keep_prob = tf.placeholder(tf.float32)

# input place holders
X = tf.placeholder(tf.float32, [None, 28, 28 ,1])
Y = tf.placeholder(tf.float32, [None, 2])  

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
# strides = [batch_size, x, y, channel ]
# L1 ImgIn shape=(?, 28, 28, 1)
# filter 3 x 3 x 1 => 32 filters
# Conv     -> (?, 28, 28, 32)
# Pool     -> (?, 14, 14, 32)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
# L2 ImgIn shape=(?, 14, 14, 32)
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)

# L3 ImgIn shape=(?, 7, 7, 64)
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
#    Conv      ->(?, 7, 7, 128)
#    Pool      ->(?, 4, 4, 128)
#    Reshape   ->(?, 4 * 4 * 128) # Flatten them for FC
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
'''
Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 2048), dtype=float32)
'''

# L4 FC 4x4x128 inputs -> 625 outputs
W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
'''
Tensor("Relu_3:0", shape=(?, 625), dtype=float32)
Tensor("dropout_3/mul:0", shape=(?, 625), dtype=float32)
'''

# L5 Final FC 625 inputs -> 10 outputs
W5 = tf.get_variable("W5", shape=[625, 2], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([2]))
logits = tf.matmul(L4, W5) + b5
'''
Tensor("add_1:0", shape=(?, 10), dtype=float32)
'''
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(Y_train) / batch_size)

    for i in range(total_batch):

        batch_xs = X_[i*batch_size:(i+1)*batch_size]
        batch_ys = Y_[i*batch_size:(i+1)*batch_size]

        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    print('Accuracy:', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test, keep_prob: 1}))

print('Learning Finished!')

# Test model and check accuracy
print('Accuracy:', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test, keep_prob: 1}))

X_train.shape() =  (7127, 28, 28, 1)
Y_train.shape() =  (7127, 2)
X_test.shape() =  (793, 28, 28, 1)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Learning started. It takes sometime.
Epoch: 0001 cost = 0.751812149
Accuracy: 0.6633039
Epoch: 0002 cost = 0.679089736
Accuracy: 0.7301387
Epoch: 0003 cost = 0.597059816
Accuracy: 0.7704918
Epoch: 0004 cost = 0.537792108
Accuracy: 0.776797
Epoch: 0005 cost = 0.510845421
Accuracy: 0.7805801
Epoch: 0006 cost = 0.486261941
Accuracy: 0.78310215
Epoch: 0007 cost = 0.477885895
Accuracy: 0.7931904
Epoch: 0008 cost = 0.473332935
Accuracy: 0.78814626
Epoch: 0009 cost = 0.454025499
Accuracy: 0.8020176
Epoch: 0010 cost = 0.454107324
Accuracy: 0.79192936
Epoch: 0011 cost = 0.445462576
Accuracy: 0.7944515
Epoch: 0012 cost = 0.437227859
Accuracy: 0.80580074
Epoch: 0013 cost = 0.431000304
Accuracy: 0.8032787
Epoch

In [11]:
print('Accuracy:', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test, keep_prob: 1}))

Accuracy: 0.8196721


In [12]:
X_X_data_normal=X_data_normal.astype("float32")
y_pred=sess.run(logits, feed_dict={X: X_X_data_normal, keep_prob: 1})
normallist=[]
for i in b:
    normallist.append(i.split('\\')[1].split('.')[0])
pd.DataFrame(y_pred[:,:],index=normallist).to_csv('./extract-from-apk-master/out/normal_iamg.csv')

In [13]:
X_X_data_malware=X_data_malware.astype("float32")
y_pred=sess.run(logits, feed_dict={X: X_X_data_malware, keep_prob: 1})
malwarelist=[]
for i in m:
    malwarelist.append(i.split('\\')[1].split('.')[0])
pd.DataFrame(y_pred[:,:],index=malwarelist).to_csv('./extract-from-apk-master/out/malware_iamg.csv')

In [14]:
X_X_data_test=injected_X_test.astype("float32")
y_pred=sess.run(logits, feed_dict={X: X_X_data_test, keep_prob: 1})
testlist=[]
for i in t:
    testlist.append(i.split('\\')[1].split('.')[0])
pd.DataFrame(y_pred[:,:],index=testlist).to_csv('./extract-from-apk-master/out/test_iamg.csv')